<a href="https://colab.research.google.com/github/Julialunna/Artificial-Intelligence/blob/main/FL-DP-PSO-SGD/FL_DP_PSO_SGD_MNIST_Clients_gradient_clipping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pega o erro de cada um e compara. A partícula com menor erro pede o gbest.

In [1]:
pip install opacus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from torchvision import datasets, transforms
import torch.nn.functional as F
import copy
import random
import csv
import torchvision
import torchvision.models as models
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager
from opacus.accountants.utils import get_noise_multiplier
from copy import deepcopy

In [3]:

class MLP(nn.Module):

    def __init__(self, device, input_size=28*28, hidden_size=256, num_classes=10):
        super(MLP, self).__init__()
        self.device = device
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 128)
        self.fc4 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()  # ReLU é reutilizado
        self.to(device)  # Move o modelo para o dispositivo especificado

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Achata o tensor de entrada
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)  # Chamada correta da camada fc4
        return x

In [4]:
# Definições dos hiperparâmetros
NUM_CLIENTES = 5
NUM_PARTICULAS = 25
NUM_RODADAS = 10
INERCIA, C1, C2 = 0.9, 0.8, 0.9
EPSILON = 6/math.sqrt(50)
DELTA = 1e-5
SENSITIVITY = 4
MAX_NORM = 2.0
SUBSET_SIZE = 12000
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'training on {DEVICE}')

# Criando o modelo global
modelo_global = MLP(DEVICE, hidden_size=256)
criterio = nn.CrossEntropyLoss()



training on cpu


In [5]:
#Seeds para reprodutibilidade
random.seed(123)
torch.manual_seed(123)
torch.cuda.manual_seed(123)


# Função para criar um subconjunto aleatório de um dataset
def create_subset(dataset, num_clients):
    indices = list(range(len(dataset)))  # Lista de todos os índices
    random.shuffle(indices)  # Embaralha os índices para aleatoriedade

    subset_size = len(indices) // num_clients  # Tamanho de cada subconjunto
    subsets = [Subset(dataset, indices[i * subset_size : (i + 1) * subset_size]) for i in range(num_clients)]

    dataloaders = [
        DataLoader(subset, batch_size=300, shuffle=False)
        for subset in subsets
    ]
    return dataloaders

    return dataloaders
def add_module_prefix(state_dict):
    new_state_dict = {}
    for key, value in state_dict.items():
        new_key = f"_module.{key}"  # Adiciona o prefixo `_module`
        new_state_dict[new_key] = value
    return new_state_dict

class Particula:
    def __init__(self, particle_id, modelo_cliente):
        self.particle_id = particle_id
        self.pesos = {f"_module.{key}": value.clone() for key, value in modelo_cliente.state_dict().items()}
        self.melhor_pesos = copy.deepcopy(self.pesos)  # pbest (melhor posição da partícula)
        self.melhor_erro = float('inf')  # Melhor erro alcançado
        self.velocidade = {name: torch.zeros_like(param) for name, param in self.pesos.items()}  # Velocidade do PSO
        self.device = modelo_cliente.device  # Dispositivo do modelo

    def atualizar_pso(self, global_best_pesos, INERCIA, C1, C2, rodada):
        """Atualiza os pesos da partícula usando a equação do PSO."""
        global_best_pesos_ajustados = global_best_pesos
        if(rodada == 0):
          global_best_pesos_ajustados = {f"_module.{key}": value for key, value in global_best_pesos.items()}
        for name in self.pesos:
            local_rand = random.random()
            global_rand = random.random()
            self.velocidade[name] = (
                INERCIA * self.velocidade[name] +
                C1 * local_rand * (self.melhor_pesos[name] - self.pesos[name]) +
                C2 * global_rand * (global_best_pesos_ajustados[name] - self.pesos[name])
            )
            self.pesos[name] += self.velocidade[name]
            ''' sigma = SENSITIVITY * torch.sqrt((2.0 * torch.log(torch.tensor(1.0 / DELTA))).clone().detach()) / EPSILON

              # Gerar ruído diretamente com a distribuição normal do PyTorch (muito mais eficiente!)
              noise = torch.normal(mean=0, std=sigma, size=self.velocidade[name].shape, device=self.device)
              self.velocidade[name] += noise
              #clipping velocity
              self.velocidade[name] = torch.clamp(self.velocidade[name], -MAX_VELOCITY, MAX_VELOCITY)'''

    def avaliar_perda(self, modelo_cliente, criterio, dados):
        """Calcula a perda da partícula no modelo do cliente."""
        #pesos_ajustados = {f"_module.{key}": value for key, value in self.pesos.items()}

        modelo_cliente.load_state_dict(self.pesos)  # Aplica os pesos da partícula no modelo do cliente
        modelo_cliente.eval()
        total_loss = 0
        device = next(modelo_cliente.parameters()).device

        with torch.no_grad():
            for inputs, labels in dados:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = modelo_cliente(inputs)
                loss = criterio(outputs, labels)
                total_loss += loss.item()

        return total_loss / len(dados)


class Cliente:
    def __init__(self, cliente_id, modelo_global, dados, num_particulas=5):
        self.cliente_id = cliente_id
        self.modelo = copy.deepcopy(modelo_global)  # Cada cliente tem seu próprio modelo
        self.dados = dados
        self.num_particulas = num_particulas
        self.particulas = []
        self.melhor_particula = None
        self.inicializar_particulas(num_particulas)
        self.otimizador = optim.Adam(self.modelo.parameters(), lr=0.009, weight_decay=1e-5)

    def inicializar_particulas(self, num_particulas):
        """Cria um conjunto de partículas associadas ao cliente."""
        self.particulas = [Particula(i, self.modelo) for i in range(num_particulas)]

    def treinar_com_pso(self, INERCIA, C1, C2, global_best_pesos, criterio, rodada):
        """Treina as partículas usando PSO e atualiza a melhor partícula local."""

        for particula in self.particulas:
            particula.atualizar_pso(global_best_pesos, INERCIA, C1, C2, rodada)
            erro = particula.avaliar_perda(self.modelo, criterio, self.dados)
            if erro < particula.melhor_erro:
                particula.melhor_erro = erro
                particula.melhor_pesos = copy.deepcopy(particula.pesos)

        self.selecionar_melhor_particula()

    def refinar_com_adam(self, criterio, EPSILON, DELTA, MAX_NORM, SENSITIVITY):
        """Refina os pesos da melhor partícula usando Adam."""
        self.modelo.load_state_dict(self.melhor_particula.melhor_pesos)
        #train_loader = DataLoader(self.dados, batch_size=32, shuffle=True)
        device = next(self.modelo.parameters()).device
        self.modelo.train()
        for i in range(2):  # 10 épocas de refinamento com Adam
          with BatchMemoryManager(data_loader=self.dados, max_physical_batch_size=300, optimizer=self.otimizador) as new_data_loader:
            for inputs, labels in new_data_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                self.otimizador.zero_grad()
                outputs = self.modelo(inputs)
                loss = criterio(outputs, labels)
                loss.backward()
                self.otimizador.step()
                '''sigma = SENSITIVITY * torch.sqrt((2.0 * torch.log(torch.tensor(1.0 / DELTA))).clone().detach()) / EPSILON
                if i==4:
                  # Percorrer todos os parâmetros do modelo e adicionar ruído aos gradientes
                  for param in self.modelo.parameters():
                      if param.grad is not None:
                          torch.nn.utils.clip_grad_norm_(self.modelo.parameters(), MAX_NORM)
                          noise = torch.normal(mean=0, std=sigma, size=param.grad.shape, device=param.grad.device)
                          param.grad += noise  # 🔹 Adiciona ruído diretamente ao gradiente'''
        epsilon = privacy_engine.get_epsilon(delta = 1e-5)
        print(f"Orçamento de privacidade (epsilon) Cliente {self.cliente_id}: {epsilon}")
        # Atualiza os pesos da melhor partícula com os pesos refinados pelo Adam
        self.melhor_particula.pesos = copy.deepcopy(self.modelo.state_dict())

    def selecionar_melhor_particula(self):
        """Seleciona a melhor partícula do cliente."""
        self.melhor_particula = min(self.particulas, key=lambda p: p.melhor_erro)


def treinar_federado(modelo_global, clientes, criterio, num_rodadas, INERCIA, C1, C2, testloader, EPSILON, DELTA, MAX_NORM, SENSITIVITY):
    """Treina os clientes localmente e sincroniza com o servidor central, validando a acurácia."""

    melhor_peso_global = copy.deepcopy(modelo_global.state_dict())  # Inicializa com o modelo global
    melhor_erro_global = float('inf')
    for rodada in range(num_rodadas):
        resultados_rodada = []


        for cliente in clientes:
            cliente.treinar_com_pso(INERCIA, C1, C2, melhor_peso_global, criterio, rodada)  # Treino com PSO
            cliente.refinar_com_adam(criterio, EPSILON, DELTA, MAX_NORM, SENSITIVITY)  # Refinamento com Adam
            erro_cliente = cliente.melhor_particula.melhor_erro  # Obtém o melhor erro do cliente
            resultados_rodada.append((cliente.cliente_id, erro_cliente))

        resultados_sorted = sorted(resultados_rodada, key=lambda x: x[1])
        top_3_results = resultados_sorted[:3]

        melhor_cliente = random.choice(top_3_results)
        melhor_cliente_id = melhor_cliente[0]
        melhor_erro_cliente = melhor_cliente[1]

        melhor_peso_global = copy.deepcopy(clientes[melhor_cliente_id].melhor_particula.pesos)
        melhor_peso_global_ajustado = {key.replace("_module.", ""): value for key, value in melhor_peso_global.items()}
        melhor_erro_global = melhor_erro_cliente

        modelo_global.load_state_dict(melhor_peso_global_ajustado)

        test_loss, test_accuracy = avaliar_modelo(modelo_global, criterio, testloader)


        print(f"Rodada {rodada+1}/{num_rodadas}: Cliente {melhor_cliente_id} enviou os pesos.")
        print(f"Erro Global Atualizado: {melhor_erro_global:.4f}")
        print(f"Teste -> Perda: {test_loss:.4f}, Acurácia: {test_accuracy:.2f}%\n")

    print("Treinamento Federado Finalizado!")

def avaliar_modelo(modelo, criterio, testloader):
    """Avalia o modelo global no conjunto de teste."""
    modelo.eval()  # Modo de avaliação

    total_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = modelo(inputs)
            loss = criterio(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total_samples += labels.size(0)

    test_loss = total_loss / len(testloader)
    test_accuracy = (correct / total_samples) * 100

    return test_loss, test_accuracy


mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True)
X_train = mnist_train.data.view(-1, 28*28).numpy()  # Flatten (Transforma 28x28 em 784)
y_train = mnist_train.targets.numpy()  # Labels

mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True)
X_test = mnist_test.data.view(-1, 28*28).numpy()  # Flatten (Transforma 28x28 em 784)
y_test = mnist_test.targets.numpy()  # Labels

# Dividir treino e teste manualmente como no Iris

# Normalizar como no Iris
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converter para tensores
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Criar datasets como no Iris
trainset = TensorDataset(X_train, y_train)
testset = TensorDataset(X_test, y_test)


trainloaders = create_subset(trainset, NUM_CLIENTES)

testloader = DataLoader(testset, batch_size=300, shuffle=False)

noise = get_noise_multiplier(target_epsilon = 3,
                             target_delta = 1e-5,
                             sample_rate = 300/12000,
                             epochs = 10)

# Criando os clientes
clientes = [Cliente(i, modelo_global, trainloaders[i], NUM_PARTICULAS) for i in range(NUM_CLIENTES)]
privacy_engine = PrivacyEngine()
for cliente in clientes:

    cliente.modelo, cliente.otimizador, cliente.dados = privacy_engine.make_private(
        module=cliente.modelo,
        optimizer=cliente.otimizador,
        data_loader=cliente.dados,  # Seu DataLoader
        noise_multiplier=noise,      # Controle o nível de ruído
        max_grad_norm=1.0,         # Clipping dos gradientes
    )

# Executando o treinamento federado
treinar_federado(modelo_global, clientes, criterio, NUM_RODADAS, INERCIA, C1, C2, testloader, EPSILON, DELTA, MAX_NORM, SENSITIVITY)


100%|██████████| 9.91M/9.91M [00:00<00:00, 53.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.97MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 12.8MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.99MB/s]
/usr/local/lib/python3.11/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/opacus/privacy_engine.py:96: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/opacus/privacy_engine.py:152: UserWarning: PrivacyEngine detected new dataset object. Was: <torch.utils.data.dataset.Subset object at 0x7ef0e8904210>, got: <torch.utils.data.dataset.Subset object at 0x7ef0

Orçamento de privacidade (epsilon) Cliente 0: 1.33968117025341
Orçamento de privacidade (epsilon) Cliente 1: 1.7632286460470201


KeyboardInterrupt: 

 '''sigma = SENSITIVITY * torch.sqrt((2.0 * torch.log(torch.tensor(1.0 / DELTA))).clone().detach()) / EPSILON
              if i==4:
                # Percorrer todos os parâmetros do modelo e adicionar ruído aos gradientes
                for param in self.modelo.parameters():
                    if param.grad is not None:
                        torch.nn.utils.clip_grad_norm_(self.modelo.parameters(), MAX_NORM)
                        noise = torch.normal(mean=0, std=sigma, size=param.grad.shape, device=param.grad.device)
                        param.grad += noise  # 🔹 Adiciona ruído diretamente ao gradiente'''